# Säätietojen poiminta, lataus ja muuntaminen (ELT)

Tämä ELT-prosessi hakee tietyn maantieteellisen paikan säätiedot Ilmatieteenlaitoksen rajapinnasta. Tiedot ladataan tietokantaan, jossa ne muokataan helpommin käsiteltävään tietomalliin.

**Huom! Osa raskaammista proseduureista on kommentoituna oletuksena, jotta vältytään turhilta kyselyiltä Ilmatieteenlaitoksen API:in ja tarpeettomilta tietokannan uudelleen latauksilta.**

Haettavien tietojen asetukset:

In [1]:
#Paikka, jonka säätiedot haetaan
input.place <- "Helsinki"

#Aikaväli, jolta säätiedot haetaan (annettu kuukausi tulee lataukseen mukaan)
input.starttime <- "2021-01"
input.endtime <- "2021-06"

## 1. Säätietojen poiminta (extract)

Poimitaan säätiedot Ilmatieteenlaitoksen API:sta. Tiedot tulevat melko suurina XML-dokumentteina, jotka tallennetaan tiedostoihin muistin säästämiseksi.

In [2]:
#install.packages("XML")
library(XML)

Warning message:
"package 'XML' was built under R version 3.6.3"

In [3]:
# Parametrin muoto: 'YYYY-MM'
# Palauttaa aikavälin jaettuna osiin dataframe:na, jossa sarakkeina starttimes ja endtimes.
# Jokainen rivi on aikaväli, joka on riittävän lyhyt Ilmatieteenlaitoksen API:lle.
common.get_apifriendly_time_intervals <- function(starttime, endtime) {
    format_starttime <- function(month, year) { return (paste0(year, "-",formatC(month, width=2, flag="0"),"-01T02:00:00Z")) }
    format_endtime <- function(month, year) { return (paste0(year, "-",formatC(month, width=2, flag="0"),"-01T01:00:00Z")) }
    create_list_of_times <- function(months, years, formater) {
        times <- c()
        for (i in 1:length(months)) {
            times <- append(times, formater(months[i], years[i]))
        }
        return (times)
    }
    
    start.month <- as.integer(substring(starttime, 6, 7))
    end.month <- as.integer(substring(endtime, 6, 7))
    start.year <- as.integer(substring(starttime, 1, 4))
    end.year <- as.integer(substring(endtime, 1, 4))
    
    if(end.year < start.year) {stop("Lopetusvuosi ei voi olla ennen aloitusvuotta.")}
    if(end.year == start.year & end.month < start.month) {stop("Lopetuaika ei voi olla ennen aloitusaikaa.")}
    
    delta.years <- end.year - start.year
    
    template.months <- rep(1:12, times=(delta.years + 2))
    starttimes.months <- template.months[start.month:(end.month+12*delta.years)]
    endtimes.months <- template.months[(start.month+1):(end.month+1+12*delta.years)]
    
    template.years <- rep(start.year:(end.year+1), each=12)
    endtimes.years <- template.years[(start.month+1):(end.month+1+12*delta.years)]
    starttimes.years <- template.years[(start.month):(end.month+12*delta.years)]
    
    starttimes <- create_list_of_times(starttimes.months, starttimes.years, format_starttime)
    endtimes <- create_list_of_times(endtimes.months, endtimes.years, format_endtime)
    
    return (data.frame(starttimes = starttimes, endtimes = endtimes))
}

common.get_filename <- function(place, starttime, endtime) {
    return (paste0("saatiedot_", place, "_", gsub(":", "", starttime), "_", gsub(":", "", endtime), ".xml"))
}

common.add_filepath <- function(filename) {
    return (paste( ".\\loaded_data\\", filename, sep=""))
}

Data täytyy poimia osissa Ilmatieteenlaitoksen API:n rajoitusten takia

In [4]:
extract <- function(times, place) {
    #Päivämäärien muoto: YYYY-MM-DDTHH24:MI:SSZ
    create_source_url <- function(place, starttime, endtime) {
        return (paste(
                    "http://opendata.fmi.fi/wfs/fin?service=WFS&version=2.0.0&request=GetFeature"
                    , "&storedquery_id=fmi::observations::weather::hourly::simple&place="
                    , place
                    , "&starttime="
                    , starttime
                    , "&endtime="
                    , endtime
                    , "&", sep=""))
    }

    save_xml_from_url <- function(url, destination_file) {
        xml.content <- xmlTreeParse(url)
        saveXML(xmlRoot(xml.content), file=destination_file)
    }
    extract.starttime <- Sys.time()
    
    for (i in 1:nrow(times)) {
        url <- create_source_url(place = place
                                , starttime = as.character(times[["starttimes"]][i])
                                , endtime = as.character(times[["endtimes"]][i]))
        destination <- common.add_filepath(common.get_filename(place
                                                                , as.character(times[["starttimes"]][i])
                                                                , as.character(times[["endtimes"]][i])))
        save_xml_from_url(url = url, destination_file = destination)
    }
    
    extract.endtime <- Sys.time() 
    print(extract.endtime - extract.starttime)
}

Proseduurin ajo on kommentoitu, jotta Ilmatieteenlaitos APIin kohdistuisi mahdollisimman vähän turhia kyselyitä.
Poistetaan kommentointi, kun halutaan ladata tiedot.

In [5]:
place <- input.place

times <- common.get_apifriendly_time_intervals(input.starttime,input.endtime)

#extract(times, place)

## 2.Säätietojen lataaminen (load)
> TODO: tallennetaan xml-dokumentit tietokantaan levyn sijaan, jolloin ne on helpommin saatavilla myöhempää käyttöä varten

---

## 3. Tietojen muuntaminen (transform)
Muunnetaan tiedot helpommin käsiteltävään tietomalliin. Tässä tarkoituksena on selväkielistää säätiedoissa olevat koodit ja muodostaa taulu, jonka jyvänä on kaikki yhden tunnin sisällä tehdyt mittaukset.

In [6]:
#install.packages("odbc")
library(odbc)
library(dplyr)

Warning message:
"package 'odbc' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Windows koneella lisättynä tietokantayhteys SQL Serveriin "ODBC Data Sources"-sovelluksesta. Tietokanta on valmiiksi luotuna.

In [7]:
data_conn <- DBI::dbConnect(odbc::odbc(), "DATASOURCE", database = "SaaDB")

### 3.1 XML-dokumentin sisältö tietokantatauluun

Luodaan tietokannan taulut

In [8]:
create_table_saatiedot_raaka <- function() {
    fields_raaka <- c('INT IDENTITY(1,1) NOT NULL', 'VARCHAR(50) NOT NULL', 'DATETIME NOT NULL', 'VARCHAR(20) NOT NULL', 'VARCHAR(20)')
    names(fields_raaka) <- c('ID', 'PAIKKA', 'AIKA', 'PARAMETRIN_NIMI', 'PARAMETRIN_ARVO')
    
    DBI::dbCreateTable(data_conn, "SAATIEDOT_RAAKA", fields_raaka)
    DBI::dbExecute(data_conn, 'ALTER TABLE SAATIEDOT_RAAKA ADD CONSTRAINT [SAATIEDOT_RAAKA_PK] PRIMARY KEY ([ID])')
    }

#Poista tarvittaessa taulu
DBI::dbExecute(data_conn, 'DROP TABLE SAATIEDOT_RAAKA')
create_table_saatiedot_raaka()

[1] 0

[1] 0

Haetaan tiedot XML-tiedostoista ja tallennetaan ne tauluun.

In [9]:
saatiedot_raaka <- DBI::dbReadTable(data_conn, "SAATIEDOT_RAAKA")
# Id generoidaan taulussa automaattisesti [IDENTITY(1,1)], joten sitä ei täytetä täällä. Poistetaan se virheiden välttämiseksi.
saatiedot_raaka <- subset(saatiedot_raaka, select=-c(ID))

In [10]:
#XML-dokumentissa käytetään nimiavaruuksia, joita tarvitaan dataa lukiessa.
namespaces <- c(wfs='http://www.opengis.net/wfs/2.0', BsWfs='http://xml.fmi.fi/schema/wfs/2.0', gml='http://www.opengis.net/gml/3.2')

In [11]:
xml.content = xmlParse(file = common.add_filepath(common.get_filename(place
                                                                , as.character(times[["starttimes"]][1])
                                                                , as.character(times[["endtimes"]][1]))))
rootnode = xmlRoot(xml.content)
print(rootnode[[8000]][[1]])

<BsWfs:BsWfsElement id="BsWfsElement.1.667.8">
  <BsWfs:Location>
    <gml:Point id="BsWfsElementP.1.667.8" srsDimension="2" srsName="http://www.opengis.net/def/crs/EPSG/0/4258">
      <gml:pos>60.17523 24.94459</gml:pos>
    </gml:Point>
  </BsWfs:Location>
  <BsWfs:Time>2021-01-28T20:00:00Z</BsWfs:Time>
  <BsWfs:ParameterName>WD_PT1H_AVG</BsWfs:ParameterName>
  <BsWfs:ParameterValue>50.0</BsWfs:ParameterValue>
</BsWfs:BsWfsElement> 


In [12]:
format_time <- function(time) {
    modified_time <- sub('T', ' ', time)
    modified_time
    } 

In [13]:
load_data_to_dbtable <- function() {
    load_starttime=Sys.time()

    for (i in 1:nrow(times)) {
        xml.content = xmlParse(file = common.add_filepath(common.get_filename(place
                                                                , as.character(times[["starttimes"]][i])
                                                                , as.character(times[["endtimes"]][i]))))
        rootnode = xmlRoot(xml.content)
        for (j in 1:xmlSize(rootnode)) {
            result.time <- format_time(xpathApply(rootnode[[j]][[1]],path='BsWfs:Time',xmlValue,namespaces=namespaces))
            result.location <- xpathApply(rootnode[[j]][[1]],path='BsWfs:Location//gml:Point//gml:pos',xmlValue,namespaces=namespaces)
            result.parametername <- xpathApply(rootnode[[j]][[1]],path='BsWfs:ParameterName',xmlValue,namespaces=namespaces)
            result.parametervalue <- xpathApply(rootnode[[j]][[1]],path='BsWfs:ParameterValue',xmlValue,namespaces=namespaces)

            saatiedot_raaka[nrow(saatiedot_raaka)+1,] <- c(paikka=result.location, aika=result.time, parametrin_nimi=result.parametername, parametrin_arvo=result.parametervalue)

            if(nrow(saatiedot_raaka) >= 1000) {
                DBI::dbAppendTable(data_conn, "SAATIEDOT_RAAKA",saatiedot_raaka)
                saatiedot_raaka <- subset(saatiedot_raaka, 1==0) 
            }
        }
    }
    DBI::dbAppendTable(data_conn, "SAATIEDOT_RAAKA",saatiedot_raaka)

    print(Sys.time() - load_starttime)
    }

#load_data_to_dbtable()

### 3.2 Tietomalli
Muunnetaan suoraan XML-dokumentista haetut tiedot tietomalliin ja selväkielistetään koodit.

#### Selitteet

|Koodi         |Selite                           |Taulun attribuutti          |Yksikkö|
|--------------|---------------------------------|----------------------------|-------|
|PA_PT1H_AV    |Ilmanpaine keskiarvo             |ilmanpaine                  |hPa    |
|PRA_PT1H_A    |Sademäärä                        |sademaara                   |mm     |
|PRI_PT1H_M    |Sateen suurin intensiteetti      |sade_intensiteetti_maksimi  |mm/h   |
|RH_PT1H_AV    |Suhteellinen kosteus keskiarvo   |suhteellinen_kosteus        |%      |
|TA_PT1H_AV    |Ilman lämpötila keskiarvo        |lampotila                   |degC   |
|TA_PT1H_MA    |Ilman lämpötila maksimi          |lampotila_maksimi           |degC   |
|TA_PT1H_MI    |Ilman lämpötila minimi           |lampotila_minimi            |degC   |
|WAWA_PT1H_    |Vallitseva sää                   |saa                         |       |
|WD_PT1H_AV    |Tuulen suunta keskiarvo          |tuulen_suunta               |deg    |
|WS_PT1H_AV    |Tuulen nopeus keskiarvo          |tuulen_nopeus               |m/s    |
|WS_PT1H_MA    |Tuulen nopeus maksimi            |tuulen_nopeus_maksimi       |m/s    |
|WS_PT1H_MI    |Tuulen nopeus minimi             |tuulen_nopeus_minimi        |m/s    |


In [14]:
create_empty_table_saatiedot <- function() {
    DBI::dbExecute(data_conn, 'DROP TABLE IF EXISTS SAATIEDOT')
    fields_raaka <- c('INT IDENTITY(1,1) NOT NULL', 'VARCHAR(50) NOT NULL', 'DATETIME NOT NULL'
                      , 'VARCHAR(10)', 'VARCHAR(10)', 'VARCHAR(10)', 'VARCHAR(10)'
                      , 'VARCHAR(10)', 'VARCHAR(10)', 'VARCHAR(10)', 'VARCHAR(10)'
                      , 'VARCHAR(10)')
    names(fields_raaka) <- c('ID', 'PAIKKA', 'AIKA'
                             , 'ILMANPAINE', 'SUHTEELLINEN_KOSTEUS', 'LAMPOTILA', 'LAMPOTILA_MAKSIMI'
                             , 'LAMPOTILA_MINIMI', 'TUULEN_SUUNTA', 'TUULEN_NOPEUS', 'TUULEN_NOPEUS_MAKSIMI'
                             , 'TUULEN_NOPEUS_MINIMI')
    
    DBI::dbCreateTable(data_conn, "SAATIEDOT", fields_raaka)
    DBI::dbExecute(data_conn, 'ALTER TABLE SAATIEDOT ADD CONSTRAINT [SAATIEDOT_PK] PRIMARY KEY ([ID])')
    }

create_empty_table_saatiedot()

[1] 0

## 4. Yksikkötestit
Täällä on omien funktioiden yksikkötestit. Testin nimi muodossa "test.funkiton_nimi".

In [15]:
test.common.get_apifriendly_time_intervals <- function() {
    #Tulos on data.frame
    print(paste0("[1] ", is.data.frame(common.get_apifriendly_time_intervals("2021-01", "2021-02"))))
    
    #Kun kuukausia on yksi
    print(paste0("[2] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-01", "2021-01")[["starttimes"]][[1]]), "2021-01-01T02:00:00Z")))
    print(paste0("[3] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-01", "2021-01")[["endtimes"]][[1]]), "2021-02-01T01:00:00Z")))
    
    #Kun kuukausia on kaksi
    print(paste0("[4] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-01", "2021-02")[["starttimes"]][[1]]), "2021-01-01T02:00:00Z")))
    print(paste0("[5] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-01", "2021-02")[["endtimes"]][[1]]), "2021-02-01T01:00:00Z")))
    print(paste0("[6] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-01", "2021-02")[["starttimes"]][[2]]), "2021-02-01T02:00:00Z")))
    print(paste0("[7] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-01", "2021-02")[["endtimes"]][[2]]), "2021-03-01T01:00:00Z")))
    
    #Kun aloituskuukausi on eri
    print(paste0("[8] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-02", "2021-02")[["starttimes"]][[1]]), "2021-02-01T02:00:00Z")))
    print(paste0("[9] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-02", "2021-03")[["starttimes"]][[1]]), "2021-02-01T02:00:00Z")))
    print(paste0("[10] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-02", "2021-03")[["endtimes"]][[1]]), "2021-03-01T01:00:00Z")))
    print(paste0("[11] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-02", "2021-03")[["starttimes"]][[2]]), "2021-03-01T02:00:00Z")))
    print(paste0("[12] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-02", "2021-03")[["endtimes"]][[2]]), "2021-04-01T01:00:00Z")))
    
    #Kun kuukausia on kolme
    print(paste0("[13] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-03", "2021-05")[["starttimes"]][[3]]), "2021-05-01T02:00:00Z")))
    print(paste0("[14] ", identical(as.vector(common.get_apifriendly_time_intervals("2021-03", "2021-05")[["endtimes"]][[3]]), "2021-06-01T01:00:00Z")))
    
    #Joukko ei ylitä viimeistä kuukautta, kun pituus vaihtelee
    print(paste0("[15] ", identical(as.vector(tail(common.get_apifriendly_time_intervals("2021-01", "2021-05"), n=1)[["endtimes"]][[1]]), "2021-06-01T01:00:00Z")))
    print(paste0("[16] ", identical(as.vector(tail(common.get_apifriendly_time_intervals("2021-01", "2021-04"), n=1)[["endtimes"]][[1]]), "2021-05-01T01:00:00Z")))
    
    #Kun vuosi on eri
    print(paste0("[17] ", identical(as.vector(common.get_apifriendly_time_intervals("2019-01", "2019-01")[["starttimes"]][[1]]), "2019-01-01T02:00:00Z")))
    print(paste0("[18] ", identical(as.vector(common.get_apifriendly_time_intervals("2019-01", "2019-01")[["endtimes"]][[1]]), "2019-02-01T01:00:00Z")))
    
    #Vuoden loppu
    print(paste0("[19] ", identical(as.vector(common.get_apifriendly_time_intervals("2019-12", "2019-12")[["endtimes"]][[1]]), "2020-01-01T01:00:00Z")))
    
    #Vuoden loppu, kun aloituskuu ei ole joulukuu
    print(paste0("[20] ", identical(as.vector(common.get_apifriendly_time_intervals("2019-11", "2019-12")[["endtimes"]][[1]]), "2019-12-01T01:00:00Z")))
    
    #Vuoden alku, kun aloituskuu on edellisenä vuonna
    print(paste0("[21] ", identical(as.vector(common.get_apifriendly_time_intervals("2019-12", "2020-02")[["starttimes"]][[2]]), "2020-01-01T02:00:00Z")))

    #Vuoden loppu, kun lopetuskuukausi on joulukuu
    print(paste0("[22] ", identical(as.vector(tail(common.get_apifriendly_time_intervals("2021-01", "2021-12"), n=1)[["endtimes"]][[1]]), "2022-01-01T01:00:00Z")))
    
    #Lopetusvuosi ennen aloitusvuotta
    success <- FALSE
    success <- tryCatch({
        common.get_apifriendly_time_intervals("2021-01", "2020-12")
        }, 
        error = function(e) {
            if (identical(geterrmessage(),"Lopetusvuosi ei voi olla ennen aloitusvuotta.")) {return (TRUE)}
        })
    print(paste0("[23] ", success))
    
    #Lopetusaika (kuukausi) ennen aloitusaikaa
    success <- FALSE
    success <- tryCatch({
        common.get_apifriendly_time_intervals("2032-05", "2032-02")
        }, 
        error = function(e) {
            if (identical(geterrmessage(),"Lopetuaika ei voi olla ennen aloitusaikaa.")) {return (TRUE)}
        })
    suppressWarnings(if(success != TRUE) {success <- FALSE})
    print(paste0("[24] ", success))
    }

test.common.get_apifriendly_time_intervals()

[1] "[1] TRUE"
[1] "[2] TRUE"
[1] "[3] TRUE"
[1] "[4] TRUE"
[1] "[5] TRUE"
[1] "[6] TRUE"
[1] "[7] TRUE"
[1] "[8] TRUE"
[1] "[9] TRUE"
[1] "[10] TRUE"
[1] "[11] TRUE"
[1] "[12] TRUE"
[1] "[13] TRUE"
[1] "[14] TRUE"
[1] "[15] TRUE"
[1] "[16] TRUE"
[1] "[17] TRUE"
[1] "[18] TRUE"
[1] "[19] TRUE"
[1] "[20] TRUE"
[1] "[21] TRUE"
[1] "[22] TRUE"
[1] "[23] TRUE"
[1] "[24] TRUE"


In [16]:
test.common.get_filename <- function() {
    print(paste0("[1] ", identical(common.get_filename("Helsinki", "2021-01-01T02:00:00Z", "2021-02-01T01:00:00Z")
                                                       , "saatiedot_Helsinki_2021-01-01T020000Z_2021-02-01T010000Z.xml")))
}
test.common.get_filename()


[1] "[1] TRUE"


In [17]:
test.common.add_filepath <- function() {
        print(paste0("[1] ", identical(common.add_filepath("filename.xml"), ".\\loaded_data\\filename.xml")))
        print(paste0("[2] ", identical(common.add_filepath("filename_with-separators.xml"), ".\\loaded_data\\filename_with-separators.xml")))
}

test.common.add_filepath()

[1] "[1] TRUE"
[1] "[2] TRUE"
